In [85]:
import numpy as np
import tls_client
import pandas as pd
import webbrowser
import requests
import re
import plotly.express as px
import seaborn as sns
from utils import merge_books, pp_delta, ud_delta

pd.set_option('display.max_columns', None)

In [11]:
# -- read in scraped data from RotoWorld

pp = pd.read_csv(r"C:\Users\mikej\Downloads\rw-prizepicks-predictions-2024-01-27.csv")         #prize picks lines
ud = pd.read_csv(r"C:\Users\mikej\Downloads\rw-underdog-predictions-2024-01-27 (1).csv")           #underdog lines

# spreads = pd.read_excel(r"C:\Users\mikej\Downloads\nfl-odds-rotowire (6).xlsx",header=1)           #game spreads and totals
# spreads = (spreads
#            [['Team','Spread','Over-Under']]
#            .rename(columns = {'Team':'team','Spread':'spread','Over-Under':'over_under'}))

# books_merged = merge_books(pp,ud)
# books_merged.to_csv('data/books_merged.csv',index=False)
# books_merged.sample(3)

In [12]:


pp = pp[['Player', 'Market Name', 'Line']].rename(columns={'Player':'player','Market Name':'market','Line':'pp_line'})#.set_index('player')
ud = ud[['Player', 'Market Name', 'Line']].rename(columns={'Player':'player','Market Name':'market','Line':'ud_line'})#.set_index('player')

In [39]:
golf = pd.merge(pp,ud).melt(id_vars=['player','market']).sort_values('player').rename(columns={'variable':'book','value':'line'}).reset_index(drop=True)
golf.head(2)

,player,market,book,line
0,Akshay Bhatia,Strokes,pp_line,72.5
1,Akshay Bhatia,Birdies or Better,pp_line,3.5


In [90]:
golf.sort_values('line')

,player,market,book,line
2,Akshay Bhatia,Birdies or Better,ud_line,3.0
43,Thomas Detry,Birdies or Better,ud_line,3.0
24,Nicolai Hojgaard,Birdies or Better,ud_line,3.5
23,Min Woo Lee,Birdies or Better,ud_line,3.5
26,Nicolai Hojgaard,Birdies or Better,pp_line,3.5
13,Keegan Bradley,Birdies or Better,pp_line,3.5
11,Keegan Bradley,Birdies or Better,ud_line,3.5
30,Sahith Theegala,Birdies or Better,pp_line,3.5
21,Min Woo Lee,Birdies or Better,pp_line,3.5
33,Sahith Theegala,Birdies or Better,ud_line,3.5


In [86]:
stats = pd.read_csv(r"C:\Users\mikej\Desktop\pga_pressure\data\clean_raw_stats.csv")
stats.head()

,tour,year,season,event_completed,event_name,event_id,player_name,dg_id,finish_pos,round_num,course_name,course_num,course_par,start_hole,teetime,round_score,sg_putt,sg_arg,sg_app,sg_ott,sg_t2g,sg_total,driving_dist,driving_acc,gir,scrambling,prox_rgh,prox_fw,great_shots,poor_shots,unique_event_id,score_to_par,loser_key
0,pga,2024,2024,2024-01-07,The Sentry,16,"Rose, Justin",6093,40.0,3,Plantation Course at Kapalua,656,73,10,9:21am,75.0,-0.591,-1.327,-1.784,-0.807,-3.918,-4.508,303.8,0.600,0.833,0.429,16.369,52.203,2.0,4.0,202416,2.0,"('Rose, Justin', '202416')"
1,pga,2024,2024,2024-01-07,The Sentry,16,"List, Luke",7960,22.0,2,Plantation Course at Kapalua,656,73,1,10:27am,70.0,-1.352,-1.963,0.534,0.255,-1.174,-2.525,324.1,0.533,0.833,0.429,33.886,30.434,4.0,5.0,202416,-3.0,"('List, Luke', '202416')"
2,pga,2024,2024,2024-01-07,The Sentry,16,"List, Luke",7960,22.0,1,Plantation Course at Kapalua,656,73,1,8:21am,67.0,1.392,0.319,0.794,-0.674,0.439,1.831,308.5,0.533,0.833,0.833,28.782,32.813,4.0,3.0,202416,-6.0,"('List, Luke', '202416')"
3,pga,2024,2024,2024-01-07,The Sentry,16,"Bhatia, Akshay",26096,14.0,4,Plantation Course at Kapalua,656,73,1,9:45am,71.0,-0.808,-0.380,-1.259,-1.892,-3.531,-4.339,311.6,0.667,0.667,0.750,24.105,28.482,1.0,3.0,202416,-2.0,"('Bhatia, Akshay', '202416')"
4,pga,2024,2024,2024-01-07,The Sentry,16,"Bhatia, Akshay",26096,14.0,3,Plantation Course at Kapalua,656,73,1,9:09am,66.0,3.427,0.091,-0.358,1.331,1.065,4.492,326.2,0.533,0.833,0.875,27.435,35.239,7.0,1.0,202416,-7.0,"('Bhatia, Akshay', '202416')"


In [92]:
stats['event_completed'] = pd.to_datetime(stats['event_completed'])

temp = stats[(stats.player_name=='Cantlay, Patrick')].sort_values(by=['event_completed','round_num'],ascending=False).reset_index(drop=True)


In [93]:
temp#.round_score

,tour,year,season,event_completed,event_name,event_id,player_name,dg_id,finish_pos,round_num,course_name,course_num,course_par,start_hole,teetime,round_score,sg_putt,sg_arg,sg_app,sg_ott,sg_t2g,sg_total,driving_dist,driving_acc,gir,scrambling,prox_rgh,prox_fw,great_shots,poor_shots,unique_event_id,score_to_par,loser_key
0,pga,2024,2024,2024-01-21,The American Express,2,"Cantlay, Patrick",15466,52.0,4,Pete Dye Stadium Course,704,72,1,10:26am,76.0,-2.417,-0.646,-3.203,-0.450,-4.299,-6.716,293.2,0.500,0.667,0.571,51.873,36.302,0.0,7.0,20242,4.0,"('Cantlay, Patrick', '20242')"
1,pga,2024,2024,2024-01-21,The American Express,2,"Cantlay, Patrick",15466,52.0,3,Pete Dye Stadium Course,704,72,1,10:31am,67.0,-0.213,-0.146,0.412,1.274,1.540,1.327,305.6,0.643,0.778,0.625,23.727,26.849,4.0,2.0,20242,-5.0,"('Cantlay, Patrick', '20242')"
2,pga,2024,2024,2024-01-21,The American Express,2,"Cantlay, Patrick",15466,52.0,2,Nicklaus Tournament Course,233,72,10,8:41am,66.0,NaN,NaN,NaN,NaN,NaN,1.750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20242,-6.0,"('Cantlay, Patrick', '20242')"
3,pga,2024,2024,2024-01-21,The American Express,2,"Cantlay, Patrick",15466,52.0,1,La Quinta Country Club,202,72,1,9:14am,64.0,NaN,NaN,NaN,NaN,NaN,3.519,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20242,-8.0,"('Cantlay, Patrick', '20242')"
4,pga,2024,2024,2024-01-07,The Sentry,16,"Cantlay, Patrick",15466,12.0,4,Plantation Course at Kapalua,656,73,1,9:09am,67.0,-0.833,-0.338,-0.154,0.986,0.494,-0.339,318.9,0.667,0.944,0.500,23.873,26.614,0.0,0.0,202416,-6.0,"('Cantlay, Patrick', '202416')"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480,pga,2017,2017,2017-03-12,Valspar Championship,475,"Cantlay, Patrick",15466,2.0,1,Innisbrook Resort (Copperhead),665,71,1,9:51am,71.0,-0.705,-0.823,1.010,0.962,1.149,0.444,276.7,0.769,0.722,0.625,55.062,29.422,3.0,3.0,2017475,0.0,"('Cantlay, Patrick', '2017475')"
481,pga,2017,2017,2017-02-12,AT&T Pebble Beach Pro-Am,5,"Cantlay, Patrick",15466,48.0,4,Pebble Beach GL,5,72,10,8:22am,72.0,-0.097,-0.693,-1.336,0.789,-1.240,-1.338,278.8,0.643,0.556,0.545,27.378,37.355,1.0,4.0,20175,0.0,"('Cantlay, Patrick', '20175')"
482,pga,2017,2017,2017-02-12,AT&T Pebble Beach Pro-Am,5,"Cantlay, Patrick",15466,48.0,3,Pebble Beach GL,5,72,10,12:12pm,71.0,2.227,-0.838,-2.255,0.690,-2.403,-0.176,265.0,0.857,0.722,0.500,135.839,31.402,2.0,5.0,20175,-1.0,"('Cantlay, Patrick', '20175')"
483,pga,2017,2017,2017-02-12,AT&T Pebble Beach Pro-Am,5,"Cantlay, Patrick",15466,48.0,2,Spyglass Hill GC,205,72,1,11:14am,71.0,NaN,NaN,NaN,NaN,NaN,-0.471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20175,-1.0,"('Cantlay, Patrick', '20175')"


In [94]:
px.scatter(temp.round_score[:20]).add_hline(y=71)